# 4. Pesquisa das metodologias
- Filtra conteúdo de ESG segundo palavras-chave
- Envia trechos filtrados à análise da API da OpenAI
- Sintetiza ao arquivo de metadados informações sobre origem, objetivo, método e referência da metodologia ESG e as consolida em keys específicas
- Cria arquivo dados.json

# Filtra e pesquisa informações sobre a metodologia ESG referidas nos regulamentos

In [1]:
import sys
sys.path.append('../src')
import filtro, json
import random
import analise
import pandas as pd

## Filtra informações sobre objetivos e metodologias ESG dos fundos.
- Seleciona trechos que contêm os objetivos dos fundos e as metodologias ESG empregadas.
- Registra novo arquivo entre os metadados.

In [2]:
with open('../data/json/metadados.json', 'r', encoding='utf-8') as f:
        metadados = json.load(f)


In [3]:
for fundo in metadados:
    try:
        filtro.metodologias(fundo) # Retorna metadados processados segundo filtros
    except KeyError:
        print(f"Erro. Conferir metadados do fundo {fundo}")

Arquivo processado: Aviva IGC ESG IA FC FI Mult Cred Priv IE.txt
Arquivo processado: BNPP M.CRED FI RF CRED PRIV LP SUST IS.txt
Arquivo processado: ITAÚ INDEX ESG ÁGUA AÇÕES FX IE FC DE IS.txt
Arquivo processado: SCHRODER BEST IDEAS ESG FIA IS.txt
Arquivo processado: SCH SUST AÇÕES GLOB USD FC FIA IS IE.txt
Arquivo processado: BB AÇÕES EQUIDADE IS FI.txt
Arquivo processado: BRA P ES EQ GLO FC FI MULT PGBL-VGBL.txt
Arquivo processado: BT P CRED CORP ESG FI RF IS CRED PRIV LP.txt
Arquivo processado: JGP ESG FC FIA.txt
Arquivo processado: JGP ESG FC FIA - FEEDER I.txt
Arquivo processado: VOX DESENVOL SUST ICATU PREV IS FI RF CRED PRIV.txt
Arquivo processado: CARTEIRA ITAÚ ESG INT FI MULT IE.txt
Arquivo processado: JGP ESG INST FC FIA - FEEDER VII.txt
Arquivo processado: IT NOW ISE FUNDO DE INDICE IS.txt
Arquivo processado: CSHG MASTER PORT ESG CRED PRIV FI MULT.txt
Arquivo processado: BRAM FI RF CRED PRIV IS SUSTENTABILIDADE.txt
Arquivo processado: JGP EQUILÍBRIO FIF IS RF CRED PRIV LP RE

In [4]:
random.sample(metadados, 15) # Confere integridade da lista

[{'CNPJ': '51.681.561/0001-34',
  'Código ANBIMA': '729620',
  'Patrimônio Líquido': 'R$ 27.941.303,42',
  'Taxa de administração ao ano': '0,80%',
  'Primeiro aporte': '25/09/2023',
  'Classe ANBIMA': 'Ações',
  'Tipo ANBIMA': 'Ações Indexados',
  'Classe CVM': 'Ações',
  'Sub Classe CVM': 'Não se Aplica',
  'Fundo ESG': 'Sim',
  'Composição do fundo': 'FIC',
  'Fundo Master': 'Não',
  'Estrutura de gestão': 'Feeder',
  'Benchmark': 'OUTROS',
  'Foco de atuação': 'Não se aplica',
  'Investimento no exterior': 'Não se Aplica',
  'Credito privado': 'Não',
  'Tipo de investidor': 'Não Há Restrição',
  'Característica do investidor': 'Geral',
  'Cota de abertura': 'Não',
  'Fundo permite alavancagem': 'Não',
  'Aberto estatutariamente': 'Sim',
  'Tributação Alvo': 'Renda Variável',
  'Permite aplicação automática': 'Não',
  'Moeda': 'R$',
  'Administrador': 'BB ASSET MANAGEMENT',
  'Gestor': 'BB ASSET MANAGEMENT',
  'Distribuidor': 'BANCO DO BRASIL',
  'Custodiante': 'BANCO DO BRASIL',
  

In [5]:
with open('../data/json/metadados.json', 'w', encoding='utf-8') as f: # Salvando os metadados atualizados em um arquivo JSON ao final do processamento
    json.dump(metadados, f, ensure_ascii=False, indent=4)

## Analisa trechos selecionados por meio da API da OpenAI

In [6]:
prompt = """No documento estão trechos relacionados a fundos de investimento. Para cada trecho, realize as seguintes tarefas:

1. Identifique a origem da metodologia ASG, se "interna" ou "externa".
2. Sumarize o objetivo ASG do fundo. Se não houver, registre como "n/d".
3. Categorize o objetivo ASG em "geral" ou "específico" (e.g., água, redução das emissões de carbono, mudanças climáticas etc). Se não houver, registre como "n/d".
4. Descreva os métodos que o fundo emprega para selecionar ativos. Use os seguintes métodos: "filtro negativo" ("negative screening"), "filtro baseado em normas"("norms-based screening"), "filtro positivo" ("best-in-class"), "investimento temático" (seleção temática, sustainability-themed investment), "integração ESG" (ESG integration, incorporação de fatores ESG, integrar, incorporar), "engajamento" (stewardship, stakeholder engagement, shareholder engagement, proxy voting/voto à distância), "investimentos de impacto" (impact investing, community impact investment) ou "outros". Caso haja mais de um método, use uma lista. Se não houver nenhuma referência a métodos, responda com "n/d".
5. Diga se o fundo emprega algum índice ou agência de rating externo (ISE B3, MSCI, IFIX, S&P). Se não houver, registre como "n/d".
6. Identifique a principal fonte ou referência da metodologia ASG, utilizando NER (Named-Entity Recognition) ou outra técnica de NLP. Se não houver, registre como "n/d".
7. Extraia a URL da referência. Se não houver, registre como "n/d".
8. Verifique se o regulamento do fundo menciona a elaboração de relatório sobre os resultados ASG, respondendo "Sim" ou "Não". Tenha atenção: o relatório deve ser sobre os objetivos ASG do fundo, não um relatório financeiro genérico, a não ser que este tenha uma seção dedicada aos objetivos ASG.

As respostas devem ser estruturadas como um dicionário Python válido com as chaves: "Origem Metodologia ASG", "Objetivo ASG", "Método ASG", "Referência ASG", "URL Referência ASG", "Relatório ASG". Se houver mais de uma resposta, inclua-as em uma lista dentro do dicionário. O dicionário todo deve ser conciso e não exceder 300 tokens.

Utilize palavras-chave específicas para ajudar na pesquisa:

- Para ASG: "verde", "responsável", "melhores práticas", "esg", "asg", "positiva", "raça", "diversidade", "governança", "ambiente", "equidade", "sociais", "gênero", "sustentável", "sustentabilidade", "transição energética", "carbono", "água", "energia", "trabalho escravo", "trabalho infantil", "mudanças climáticas", "biodiversidade", "desmatamento", "eficiência energética", "recursos naturais", "limpa", "poluição", "inclusão", "direitos humanos", "ética", "transparência".

- Para o campo "objetivo": "objetivo do fundo", "estratégia", "compromisso", "política de investimento", "benefício".

- Para o campo "método": "metodologia", "critérios", "método", "abordagem", "avaliação", "índice", "filtro", "parâmetros", "métrica", "índice".

- Para o campo "referências": "princípio", "princípios", "diretrizes", "normas", "formulário", "regra", "decreto", "lei", "taxonomia".

- Para o campo "relatório": "relatório", "periódico", "periodicidade", "parecer", "resultados", "transparência", "diligência", "demonstrativo", "due diligence".

Inclua "dúvida" ao final de substrings incertas. Exemplos de como as respostas devem ser formatadas:

- Trecho completo: "A GESTORA possui uma Estratégia de Sustentabilidade que descreve seus objetivos e metodologia de investimento sustentável, em linha com as Regras e Procedimentos para Investimentos em Ativos Sustentáveis publicada pela Anbima."
  Resposta: {"Origem Metodologia ASG": "interna", "Objetivo ASG": "n/d", "Objetivo Geral/Específico": "n/d", "Método ASG": "n/d", "Índice": "n/d", "Referência ASG": "Anbima", "URL Referência ASG": "n/d", "Relatório ASG": "não"}

- Trecho completo: "O FUNDO tem como OBJETIVO promover a equidade de gênero e proporcionar retornos absolutos no médio e longo prazo, através de investimentos realizados em ativos de renda variável selecionados e de engajamento junto às empresas elegíveis."
  Resposta: {"Origem Metodologia ASG": "interna", "Objetivo ASG": "promover a equidade de gênero", "Objetivo Geral/Específico": "específico", "Método ASG": ["investimentos temáticos", "engajamento"], "Índice": "n/d", "Referência ASG": "n/d", "URL Referência ASG": "n/d", "Relatório ASG": "não"}

- Trecho completo: "O Formulário referente a metodologia ASG do FUNDO pode ser
    encontrada no site da GESTORA, conforme 'link' de acesso abaixo:
    https://www.cognitoforms.com/ANBIMA/FormulárioMetodologiaASG#RVLP8HQMqu
    U4iPNE9euhb3PY7M1jYt-QrtI2x4jKuGQ$*."
    Resposta: {"Origem Metodologia ASG": "interna", "Objetivo ASG": "n/d", "Objetivo Geral/Específico": "n/d", "Método ASG": "n/d", "Índice": "n/d", "Referência ASG": "n/d", "URL referência ASG": "https://www.cognitoforms.com/ANBIMA/FormulárioMetodologiaASG#RVLP8HQMqu
     U4iPNE9euhb3PY7M1jYt-QrtI2x4jKuGQ$*", "Relatório ASG": "não"}  

A resposta deve ser precisa e estruturada rigorosamente como um dicionário Python para facilitar a análise e o processamento dos dados.


    """

In [7]:
erros = []
respostas_dict = {}  # Dicionário para coletar os resultados

for fundo in metadados:
    try:
        resultado_analise = analise.analisar_esg(fundo, prompt)
        resultado_analise = eval(resultado_analise)  # Converte a string de resposta em um dicionário
        respostas_dict[fundo['Fundo']] = resultado_analise
    except Exception as e:
        print(f"Erro ao analisar o fundo {fundo['Fundo']}: {e}")
        erros.append(fundo['Fundo'])  # Grava o nome do fundo em caso de erro
        continue
    print(f"Fundo {fundo['Fundo']} analisado.")

# Convertendo o dicionário de respostas em um DataFrame
respostas = pd.DataFrame.from_dict(respostas_dict, orient='index')


Fundo Aviva IGC ESG IA FC FI Mult Cred Priv IE analisado.
Fundo BNPP M.CRED FI RF CRED PRIV LP SUST IS analisado.
Fundo ITAÚ INDEX ESG ÁGUA AÇÕES FX IE FC DE IS analisado.
Fundo SCHRODER BEST IDEAS ESG FIA IS analisado.
Fundo SCH SUST AÇÕES GLOB USD FC FIA IS IE analisado.
Fundo BB AÇÕES EQUIDADE IS FI analisado.
Fundo BRA P ES EQ GLO FC FI MULT PGBL/VGBL analisado.
Fundo BT P CRED CORP ESG FI RF IS CRED PRIV LP analisado.
Fundo JGP ESG FC FIA analisado.
Fundo JGP ESG FC FIA - FEEDER I analisado.
Fundo VOX DESENVOL SUST ICATU PREV IS FI RF CRED PRIV analisado.
Fundo CARTEIRA ITAÚ ESG INT FI MULT IE analisado.
Fundo JGP ESG INST FC FIA - FEEDER VII analisado.
Fundo IT NOW ISE FUNDO DE INDICE IS analisado.
Fundo CSHG MASTER PORT ESG CRED PRIV FI MULT analisado.
Fundo BRAM FI RF CRED PRIV IS SUSTENTABILIDADE analisado.
Fundo JGP EQUILÍBRIO FIF IS RF CRED PRIV LP RESP LIMITADA analisado.
Fundo BLAC ESG GL BON INC IE FIC FI MULT AGORA analisado.
Fundo BB AÇÕES FX NORD GLOB DIS ESG PVT IE FC

In [8]:
erros

['ITAU GOVERNANÇA CORPORATIVA ACOES FI IS',
 'ITAÚ ESG H2O AÇÕES IE FI IS',
 'SANTANDER ETHICAL AÇÕES SUST IS FI',
 'BNPP FC Fdos Incent Inv Infra RF Sustentavel IS',
 'Blackrock FIA Globais ESG IE',
 'BRAD FI RF CRED PRIV LP IS PERF SRI 20',
 'BRAD MULTI GLOBAL EQ ESG USD FIA IE',
 'JGP ESG PREVIDENCIÁRIO MASTER FIA IS',
 'ITAÚ MOMENTO ESG AÇÕES FC IS']

In [9]:
for fundo in metadados:
    if fundo["Fundo"] in erros:
        try:
            resultado_analise = analise.analisar_esg(fundo, prompt)
            resultado_analise = eval(resultado_analise)  # Converte a string de resposta em um dicionário
            respostas_dict[fundo['Fundo']] = resultado_analise
        except Exception as e:
            print(f"Erro ao analisar o fundo {fundo['Fundo']}: {e}")
            erros.append(fundo['Fundo'])  # Grava o nome do fundo em caso de erro
            continue
        print(f"Fundo {fundo['Fundo']} analisado.")

Fundo ITAU GOVERNANÇA CORPORATIVA ACOES FI IS analisado.
Erro ao analisar o fundo ITAÚ ESG H2O AÇÕES IE FI IS: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_f7728d77dedcabb8e592e8b880c75c11 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}
Fundo SANTANDER ETHICAL AÇÕES SUST IS FI analisado.
Fundo BNPP FC Fdos Incent Inv Infra RF Sustentavel IS analisado.
Fundo Blackrock FIA Globais ESG IE analisado.
Fundo BRAD FI RF CRED PRIV LP IS PERF SRI 20 analisado.
Fundo BRAD MULTI GLOBAL EQ ESG USD FIA IE analisado.
Fundo JGP ESG PREVIDENCIÁRIO MASTER FIA IS analisado.
Fundo ITAÚ MOMENTO ESG AÇÕES FC IS analisado.


In [10]:
for fundo in metadados:
    if fundo["Fundo"] == 'ITAÚ ESG H2O AÇÕES IE FI IS':
        try:
            resultado_analise = analise.analisar_esg(fundo, prompt)
            resultado_analise = eval(resultado_analise)  # Converte a string de resposta em um dicionário
            respostas_dict[fundo['Fundo']] = resultado_analise
        except Exception as e:
            print(f"Erro ao analisar o fundo {fundo['Fundo']}: {e}")
            erros.append(fundo['Fundo'])  # Grava o nome do fundo em caso de erro
            continue
        print(f"Fundo {fundo['Fundo']} analisado.")

Fundo ITAÚ ESG H2O AÇÕES IE FI IS analisado.


In [11]:
with open('../data/json/respostas.json', 'w', encoding='utf-8') as f: # Salva o progresso em dados.json.
    json.dump(respostas_dict, f, ensure_ascii=False, indent=4)

In [12]:
len(respostas_dict)

149

In [13]:
len(metadados)

149